In [2]:
from helpers import *
import numpy as np
import pandas as pd
import os
import pickle
from LDA_helpers import process_tweets
import re
from pre_process_text import clean
import gensim
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/benjaminfuhrer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
DATA_PATH = access_folder()
GENERATED_DATA_PATH = access_folder('generated')
print(os.listdir(DATA_PATH))
print(os.listdir(GENERATED_DATA_PATH))

['IRAhandle_tweets_4.csv', 'IRAhandle_tweets_5.csv', 'IRAhandle_tweets_7.csv', 'IRAhandle_tweets_6.csv', 'IRAhandle_tweets_2.csv', '.DS_Store', 'IRAhandle_tweets_3.csv', 'IRAhandle_tweets_1.csv', '.gitignore', 'IRAhandle_tweets_8.csv', 'IRAhandle_tweets_9.csv']
['cleaned_tweets_by_authors.txt', 'lda_model.zip', 'hash_lda_model', 'line_to_author.pickle', 'hash_lda_model.state.sstats.npy', 'cleaned_tweets_by_hashtag.txt', 'lda_model.id2word', 'hash_lda_model.state', 'lda_model', 'lda_model.expElogbeta.npy', 'line_to_hash.pickle', 'hash_lda_model.id2word', 'lda_model.state.sstats.npy', '.gitignore', 'tweets_by_hashtag.txt', 'tweet_author_df.pickle', 'hash_lda_model.expElogbeta.npy', 'lda_model.state', 'hashtags.txt', 'hash_lda_model.zip', 'tweet_hash_df.pickle', 'tweets_by_authors.txt']


In [3]:
# loading all data
df_s = []
for file in os.listdir(DATA_PATH):
    if file[-4:] == '.csv':
        df_s.append(pd.read_csv(DATA_PATH + file))
tweets = pd.concat(df_s)
tweets

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1.674084e+09,GAB1ALDANA,People are too toxic. I think I have people po...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2150,RETWEET,Hashtager,0,1,HashtagGamer
1,1.674084e+09,GAB1ALDANA,#NowPlaying Don't Shoot (I'm a Man) by @DEVO -...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2146,RETWEET,Hashtager,0,1,HashtagGamer
2,1.674084e+09,GAB1ALDANA,the 'I'm the most boring person in the world' ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2159,RETWEET,Hashtager,0,1,HashtagGamer
3,1.674084e+09,GAB1ALDANA,#MyAchillesHeel slippery floors https://t.co/R...,United States,Norwegian,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2160,RETWEET,Hashtager,0,1,HashtagGamer
4,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Boring narcissists.....nothing...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2158,RETWEET,Hashtager,0,1,HashtagGamer
5,1.674084e+09,GAB1ALDANA,Your opinion on Hillary really matters to a no...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2014,2154,RETWEET,Hashtager,0,1,HashtagGamer
6,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Lilith from Frasier. 0_O,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2156,RETWEET,Hashtager,0,1,HashtagGamer
7,1.674084e+09,GAB1ALDANA,Come find us at the national mall for the #DC ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2157,RETWEET,Hashtager,0,1,HashtagGamer
8,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Trolling celebrities. I have...,United States,English,7/30/2016 20:17,7/30/2016 20:17,3395,2013,2161,RETWEET,Hashtager,0,1,HashtagGamer
9,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Morbid comedy �,United States,English,7/30/2016 20:17,7/30/2016 20:17,3395,2013,2167,RETWEET,Hashtager,0,1,HashtagGamer


In [4]:
# only keeping tweets in English
english_tweets = tweets.loc[tweets['language'] == 'English']
english_tweets.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1.674084e+09,GAB1ALDANA,People are too toxic. I think I have people po...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2150,RETWEET,Hashtager,0,1,HashtagGamer
1,1.674084e+09,GAB1ALDANA,#NowPlaying Don't Shoot (I'm a Man) by @DEVO -...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2146,RETWEET,Hashtager,0,1,HashtagGamer
2,1.674084e+09,GAB1ALDANA,the 'I'm the most boring person in the world' ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2159,RETWEET,Hashtager,0,1,HashtagGamer
4,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Boring narcissists.....nothing...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2158,RETWEET,Hashtager,0,1,HashtagGamer
5,1.674084e+09,GAB1ALDANA,Your opinion on Hillary really matters to a no...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2014,2154,RETWEET,Hashtager,0,1,HashtagGamer


In [ ]:
# groupby author
txt_path = GENERATED_DATA_PATH + 'tweets_by_authors.txt'
author_group = english_tweets.groupby('author')
line_to_author_dictionary = dict()
index = 0
with open(txt_path, 'w') as f:
    for name, group in author_group:
        line = ''
        for content in group.content.values:
            line = line + str(content)
        f.write(line+'\n')
        line_to_author_dictionary[index] = name
        index += 1
# save dictionary to pickel file
dict_path = DATA_PATH + 'line_to_author.pickle'
save('line_to_author', line_to_author_dictionary, folder_name='data')

In [11]:
hashtag_list = []
for content in english_tweets.content.values:
    hashtags = list(set([re.sub(r"(\W+)$", "", j) for j in set([i for i in str(content).split() if i.startswith("#")])]))
    hashtag_list.append(hashtags)
english_tweets['Hash_tags'] = hashtag_list
english_tweets.head(5)

/anaconda3/lib/python3.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category,Hash_tags
0,1.674084e+09,GAB1ALDANA,People are too toxic. I think I have people po...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2150,RETWEET,Hashtager,0,1,HashtagGamer,[]
1,1.674084e+09,GAB1ALDANA,#NowPlaying Don't Shoot (I'm a Man) by @DEVO -...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2146,RETWEET,Hashtager,0,1,HashtagGamer,[#NowPlaying]
2,1.674084e+09,GAB1ALDANA,the 'I'm the most boring person in the world' ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2159,RETWEET,Hashtager,0,1,HashtagGamer,[#PokemonGO]
4,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Boring narcissists.....nothing...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2158,RETWEET,Hashtager,0,1,HashtagGamer,[#MyAchillesHeel]
5,1.674084e+09,GAB1ALDANA,Your opinion on Hillary really matters to a no...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2014,2154,RETWEET,Hashtager,0,1,HashtagGamer,[]


In [13]:
hash_tweets = english_tweets.Hash_tags.apply(pd.Series).stack()
#.reset_index(level=2, drop=True).to_frame('Hash_tags')
hash_tweets

1      0                 #NowPlaying
2      0                  #PokemonGO
4      0             #MyAchillesHeel
6      0             #MyAchillesHeel
7      0                         #DC
       1                   #PokeWalk
       2                    #Pokemon
8      0                        #NRA
       1             #MyAchillesHeel
9      0             #MyAchillesHeel
10     0             #MyAchillesHeel
11     0             #MyAchillesHeel
12     0             #MyAchillesHeel
13     0             #MyAchillesHeel
14     0                      #MyBad
       1       #TimeTravelerProblems
15     0       #TimeTravelerProblems
16     0        #NameYourNaughtyBits
17     0          #HowYouWonThemOver
18     0                #SpliffFilms
19     0       #TimeTravelerProblems
20     0                #SpliffFilms
21     0                      #quote
22     0    #takeasongaroundtheworld
23     0       #TimeTravelerProblems
25     0                #SpliffFilms
26     0       #UnitedStatesIn3Words
2

In [20]:
english_tweets_dup = english_tweets.join(hash_tweets.reset_index(level=1, drop=True).to_frame('Hash_tags_dup'))

In [22]:
# groupby hashtag
txt_hash_path = GENERATED_DATA_PATH  + 'tweets_by_hashtag.txt'
hash_group = english_tweets_dup.groupby('Hash_tags_dup')
line_to_hash_dictionary = dict()
index = 0
with open(txt_hash_path, 'w') as f:
    for name, group in hash_group:
        line = ''
        for content in group.content.values:
            line = line + str(content)
        f.write(line+'\n')
        line_to_hash_dictionary[index] = name
        index += 1
# save dictionary to pickel file
save('line_to_hash', line_to_hash_dictionary, folder_name='data')

In [3]:
#clean hash
txt_hash_path = GENERATED_DATA_PATH  + 'tweets_by_hashtag.txt'
cleaned_hash_txt_path = GENERATED_DATA_PATH + 'cleaned_tweets_by_hashtag.txt'
f = open(txt_hash_path)
with open(cleaned_hash_txt_path, 'w') as c_f:
    for line in f:
        c_f.write(clean(line)+'\n')
    f.close()

In [4]:
# clean author
txt_path = GENERATED_DATA_PATH  + 'tweets_by_authors.txt'
cleaned_txt_path = GENERATED_DATA_PATH  + 'cleaned_tweets_by_authors.txt'
f = open(txt_path)
with open(cleaned_txt_path, 'w') as c_f:
    for line in f:
        c_f.write(clean(line)+'\n')
    f.close()

In [5]:
# load as pandas
txt_path = GENERATED_DATA_PATH  + 'tweets_by_authors.txt'
cleaned_txt_path = GENERATED_DATA_PATH  + 'cleaned_tweets_by_authors.txt'
cleaned_data = pd.read_csv(cleaned_txt_path, header=None)
cleaned_data.columns = ['Tweet']
cleaned_data.head(10)

,Tweet
0,sitting democrat senator trial corruption bare...
1,foodpoisoning joke walmart kochfarms god turke...
2,someone even petition watch bleeding thug shoo...
3,come vedere streaming campo juventusmilan live...
4,follow half onion mage america great follow an...
5,russia session syria life putting love trump i...
6,offense start carr raidernation turn tempo rai...
7,blacklivesmatter blm equality equalrights pay ...
8,world leader please wake putinpeacemaker isi b...
9,isi oil tanker aare attacked russian air force...


In [6]:
# load dictionary
line_to_author_dict = load('line_to_author')
# add authors as column
cleaned_data['Authors'] = [line_to_author_dict[key] for key in cleaned_data.index.values]
# save it
cleaned_data.to_pickle(GENERATED_DATA_PATH + 'tweet_author_df.pickle')

In [7]:
# tokenize tweets
cleaned_data['Tweet'] = cleaned_data['Tweet'].apply(lambda x: x.split(" "))

In [8]:
# create dictionary of all words in all tweets
dictionary = gensim.corpora.Dictionary(cleaned_data['Tweet'])

In [9]:
# filter out tokens
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
# create bowcorpus
bow_corpus = [dictionary.doc2bow(doc) for doc in cleaned_data['Tweet']]

In [10]:
# create tf-idf model
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [11]:
# create lda_model'
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=2)

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_tfidf, dictionary)
vis

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.017742  0.005556       1        1  48.100208
13     0.012774 -0.007477       2        1  26.596275
10     0.000916  0.000138       3        1   2.752011
6      0.001767 -0.000271       4        1   2.524014
9     -0.000077 -0.000463       5        1   2.032248
3     -0.001843  0.001031       6        1   1.927621
17     0.000278  0.000425       7        1   1.621920
18    -0.000950  0.000706       8        1   1.508623
16    -0.000529  0.000345       9        1   1.400498
12    -0.001635 -0.000499      10        1   1.287007
8     -0.002049  0.000739      11        1   1.282556
0     -0.002076 -0.000489      12        1   1.155366
15    -0.002472 -0.000165      13        1   1.094694
4     -0.002608 -0.000029      14        1   1.069753
5     -0.002248  0.000050      15        1   1.062536
11    -0.002419  0.000457      16        1   1.006132
19    -0.002880  0.000285      17        1   0.973426
7     -0.003435 -0.000248      18        1   0.965219
2     -0.004535  0.000078      19        1   0.835047
1     -0.003720 -0.000169      20        1   0.804843, topic_info=       Category       Freq           Term      Total  loglift  logprob
term                                                                  
1390    Default   2.000000      kochfarms   2.000000  30.0000  30.0000
46540   Default   3.000000     demndebate   3.000000  29.0000  29.0000
3746    Default   3.000000       follower   3.000000  28.0000  28.0000
1234    Default  10.000000          trump  10.000000  27.0000  27.0000
1619    Default   3.000000         russia   3.000000  26.0000  26.0000
6920    Default   4.000000           week   4.000000  25.0000  25.0000
4837    Default   3.000000        mention   3.000000  24.0000  24.0000
800     Default   4.000000            new   4.000000  23.0000  23.0000
1837    Default   2.000000        ukraine   2.000000  22.0000  22.0000
1906    Default   2.000000      gopdebate   2.000000  21.0000  21.0000
700     Default   3.000000           love   3.000000  20.0000  20.0000
1425    Default   1.000000         turkey   1.000000  19.0000  19.0000
22458   Default   1.000000     stopthegop   1.000000  18.0000  18.0000
3270    Default   2.000000      demdebate   2.000000  17.0000  17.0000
653     Default   2.000000           last   2.000000  16.0000  16.0000
858     Default   5.000000         people   5.000000  15.0000  15.0000
1368    Default   0.000000  foodpoisoning   0.000000  14.0000  14.0000
1990    Default   3.000000           tcot   3.000000  13.0000  13.0000
4988    Default   1.000000          music   1.000000  12.0000  12.0000
7587    Default   0.000000           usda   0.000000  11.0000  11.0000
12779   Default   1.000000         flimsy   1.000000  10.0000  10.0000
1013    Default   1.000000        russian   1.000000   9.0000   9.0000
1244    Default   2.000000        twitter   2.000000   8.0000   8.0000
865     Default   1.000000          photo   1.000000   7.0000   7.0000
11301   Default   2.000000          pjnet   2.000000   6.0000   6.0000
1640    Default   1.000000          syria   1.000000   5.0000   5.0000
818     Default   3.000000          obama   3.000000   4.0000   4.0000
125     Default   5.000000          black   5.000000   3.0000   3.0000
22410   Default   1.000000        belarus   1.000000   2.0000   2.0000
287     Default   3.000000            day   3.000000   1.0000   1.0000
...         ...        ...            ...        ...      ...      ...
15862   Topic20   0.002784        flexing   0.045784   2.0222 -11.4624
94002   Topic20   0.006087        помощью   0.100989   2.0134 -10.6801
6762    Topic20   0.004727       unlawful   0.088851   1.8886 -10.9329
104224  Topic20   0.002523       westboro   0.042326   2.0025 -11.5606
53106   Topic20   0.002952         amidst   0.059061   1.8263 -11.4036
7480    Topic20   0.007502    tweetdelete   0.453089   0.7213 -10.4711


In [13]:
# load as pandas
txt_hash_path = GENERATED_DATA_PATH + 'tweets_by_hashtag.txt'
cleaned_hash_txt_path = GENERATED_DATA_PATH  + 'cleaned_tweets_by_hashtag.txt'
cleaned_hash_data = pd.read_csv(cleaned_txt_path, header=None)
cleaned_hash_data.columns = ['Tweet']
cleaned_hash_data.head(10)

,Tweet
0,sitting democrat senator trial corruption bare...
1,foodpoisoning joke walmart kochfarms god turke...
2,someone even petition watch bleeding thug shoo...
3,come vedere streaming campo juventusmilan live...
4,follow half onion mage america great follow an...
5,russia session syria life putting love trump i...
6,offense start carr raidernation turn tempo rai...
7,blacklivesmatter blm equality equalrights pay ...
8,world leader please wake putinpeacemaker isi b...
9,isi oil tanker aare attacked russian air force...


In [14]:
# load dictionary
line_to_hash_dict = load('line_to_hash', folder_name='generated')
# add authors as column
cleaned_hash_data['HashTags'] = [line_to_hash_dict[key] for key in cleaned_hash_data.index.values]
# save it
cleaned_hash_data.to_pickle(GENERATED_DATA_PATH  + 'tweet_hash_df.pickle')

In [15]:
# tokenize tweets
cleaned_hash_data['Tweet'] = cleaned_hash_data['Tweet'].apply(lambda x: x.split(" "))

In [16]:
# create dictionary of all words in all tweets
hash_dictionary = gensim.corpora.Dictionary(cleaned_hash_data['Tweet'])
bow_hash_corpus = [hash_dictionary.doc2bow(doc) for doc in cleaned_hash_data['Tweet']]
tfidf_hash = models.TfidfModel(bow_hash_corpus)
corpus_tfidf_hash = tfidf_hash[bow_hash_corpus]

In [17]:
hash_lda_model = gensim.models.LdaMulticore(corpus_tfidf_hash, num_topics=20, id2word=hash_dictionary, passes=2, workers=2)

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [18]:
pyLDAvis.enable_notebook()
vis_hash = pyLDAvis.gensim.prepare(hash_lda_model, corpus_tfidf_hash, hash_dictionary)
vis_hash

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.015443 -0.001219       1        1  47.135281
13    -0.004351  0.004937       2        1  13.041494
7     -0.003688 -0.001045       3        1   8.484102
9     -0.001001  0.000347       4        1   4.367154
2     -0.000942 -0.000255       5        1   3.920768
15     0.000875 -0.000871       6        1   2.749678
11    -0.000383 -0.001065       7        1   2.195193
4     -0.000293 -0.000173       8        1   1.831643
0      0.002815 -0.000266       9        1   1.709543
16     0.001394 -0.000482      10        1   1.547860
18     0.002578 -0.000250      11        1   1.530170
3      0.001910 -0.000219      12        1   1.454666
14     0.001931  0.000064      13        1   1.399202
10     0.000989  0.000138      14        1   1.346291
1      0.002205 -0.000150      15        1   1.332367
6      0.002217  0.000395      16        1   1.242426
17     0.003136  0.000158      17        1   1.231656
5      0.002173  0.000070      18        1   1.202922
12     0.001172 -0.000366      19        1   1.148921
8      0.002707  0.000249      20        1   1.128659, topic_info=       Category       Freq               Term      Total  loglift  logprob
term                                                                      
1390    Default   2.000000          kochfarms   2.000000  30.0000  30.0000
3746    Default   2.000000           follower   2.000000  29.0000  29.0000
1619    Default   3.000000             russia   3.000000  28.0000  28.0000
6920    Default   2.000000               week   2.000000  27.0000  27.0000
1234    Default  11.000000              trump  11.000000  26.0000  26.0000
4837    Default   0.000000            mention   0.000000  25.0000  25.0000
46540   Default   1.000000         demndebate   1.000000  24.0000  24.0000
800     Default   3.000000                new   3.000000  23.0000  23.0000
1837    Default   2.000000            ukraine   2.000000  22.0000  22.0000
1425    Default   1.000000             turkey   1.000000  21.0000  21.0000
17536   Default   1.000000         megynkelly   1.000000  20.0000  20.0000
22410   Default   0.000000            belarus   0.000000  19.0000  19.0000
1368    Default   0.000000      foodpoisoning   0.000000  18.0000  18.0000
1906    Default   1.000000          gopdebate   1.000000  17.0000  17.0000
858     Default   3.000000             people   3.000000  16.0000  16.0000
22458   Default   0.000000         stopthegop   0.000000  15.0000  15.0000
7587    Default   0.000000               usda   0.000000  14.0000  14.0000
700     Default   2.000000               love   2.000000  13.0000  13.0000
1990    Default   2.000000               tcot   2.000000  12.0000  12.0000
125     Default   3.000000              black   3.000000  11.0000  11.0000
7468    Default   1.000000          sugarpova   1.000000  10.0000  10.0000
653     Default   1.000000               last   1.000000   9.0000   9.0000
1013    Default   1.000000            russian   1.000000   8.0000   8.0000
882     Default   2.000000             police   2.000000   7.0000   7.0000
1244    Default   1.000000            twitter   1.000000   6.0000   6.0000
5808    Default   0.000000               rice   0.000000   5.0000   5.0000
6515    Default   0.000000       thanksgiving   0.000000   4.0000   4.0000
3270    Default   0.000000          demdebate   0.000000   3.0000   3.0000
7423    Default   1.000000     mariasharapova   1.000000   2.0000   2.0000
1640    Default   1.000000              syria   1.000000   1.0000   1.0000
...         ...        ...                ...        ...      ...      ...
45886   Topic20   0.005075        transcanada   0.045776   2.2847 -11.2001
717983  Topic20   0.004132  myfirsttweetnorth   0.038001   2.2652 -11.4057
76853   Topic20   0.004254                jio   0.040219   2.2376 -11.3765
204615  Topic20   0.004575          racketeer   0.043482   2.2323 -11

In [ ]:
l